In [1]:
!pip install scikit-learn==1.5.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 21.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


In [2]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
from xgboost import XGBRegressor, XGBRFRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import root_mean_squared_log_error


housing_unedited = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')

In [3]:
for col in housing_unedited.columns:
  print(housing_unedited[col].value_counts())

id
1446403850    2
1825069031    2
7856400240    2
2561340020    2
1995200200    2
             ..
5315100667    1
824069188     1
2988800065    1
8648220150    1
6147650280    1
Name: count, Length: 19846, dtype: int64
date
20140623T000000    135
20140626T000000    123
20140625T000000    120
20140708T000000    117
20150427T000000    116
                  ... 
20140727T000000      1
20150527T000000      1
20150524T000000      1
20140830T000000      1
20141115T000000      1
Name: count, Length: 370, dtype: int64
bedrooms
3     9101
4     6354
2     2567
5     1478
6      251
1      179
7       36
8       12
0       11
9        6
10       3
11       1
33       1
Name: count, dtype: int64
bathrooms
2.50    5002
1.00    3552
1.75    2842
2.25    1878
2.00    1770
1.50    1346
2.75    1098
3.00     684
3.50     681
3.25     545
3.75     146
4.00     131
4.50      92
4.25      69
0.75      63
4.75      21
5.00      19
5.25      13
5.50       9
0.00       8
1.25       8
6.00       5
5.75     

In [4]:
housing = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')

# Implemented in functions in below cell
'''
# Extract year, month, day, and time of sale into separate columns and convert to numbers (rather than strings)
housing[['sale_year', 'sale_month', 'sale_day', 'sale_time']] = housing['date'].str.extract(r'(\d{4})(\d{2})(\d{2})T(\d{6})')
housing['sale_year'] = pd.to_numeric(housing['sale_year'])
housing['sale_month'] = pd.to_numeric(housing['sale_month'])
housing['sale_day'] = pd.to_numeric(housing['sale_day'])
# Don't need the garbage format date column, throw it away
housing.drop(columns=['date'], errors='ignore', inplace=True)

# Get ratio of living space to lot space for individual house and neighboring houses and add to new columns
housing['sqft_living_vs_lot']           = housing['sqft_living']   / housing['sqft_lot']
housing['sqft_living_vs_lot_neighbors'] = housing['sqft_living15'] / housing['sqft_lot15']

# Get ratio of living and lot space compared to neighboring houses and add to new columns
housing['sqft_living_vs_neighbors'] = housing['sqft_living'] / housing['sqft_living15']
housing['sqft_lot_vs_neighbors']    = housing['sqft_lot']    / housing['sqft_lot15']
housing['sqft_living_vs_lot_vs_neighbors'] = housing['sqft_living_vs_lot'] / housing['sqft_living_vs_lot_neighbors']

# Determine last renovation, or build date if not renovated
housing['last_update_year'] = np.where(housing['yr_renovated'] > 0, housing['yr_renovated'], housing['yr_built'])
# Calculate years since last renovation ("update" from above)
housing['years_since_renovation'] = housing['sale_year'] - housing['last_update_year']
# Don't need the intermediate column, throw it away
housing.drop(columns=['last_update_year'], errors='ignore', inplace=True)
'''

###

'''
Notes:
- Not all of the IDs seem to be in the same format. More info in next cell.
- Some houses (but not all) that have 1 floor have some sqft aboveground and some sqft below.
'''
housing.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,1565930130,20141104T000000,4,3.25,3760,4675,2.0,0,0,3,...,2740,1020,2007,0,98038,47.3862,-122.048,3280,4033,429900.0
1,3279000420,20150115T000000,3,1.75,1460,7800,1.0,0,0,2,...,1040,420,1979,0,98023,47.3035,-122.382,1310,7865,233000.0
2,194000575,20141014T000000,4,1.00,1340,5800,1.5,0,2,3,...,1340,0,1914,0,98116,47.5658,-122.389,1900,5800,455000.0
3,2115510160,20141208T000000,3,1.75,1440,8050,1.0,0,0,3,...,1440,0,1985,0,98023,47.3187,-122.390,1790,7488,258950.0
4,7522500005,20140815T000000,2,1.50,1780,4750,1.0,0,0,4,...,1080,700,1947,0,98117,47.6859,-122.395,1690,5962,555000.0


In [5]:
def split_sale_column(dataframe):
  df = dataframe.copy()
  # Extract year, month, day, and time of sale into separate columns and convert to numbers (rather than strings)
  df[['sale_year', 'sale_month', 'sale_day', 'sale_time']] = df['date'].str.extract(r'(\d{4})(\d{2})(\d{2})T(\d{6})')
  df['sale_year'] = pd.to_numeric(df['sale_year'])
  df['sale_month'] = pd.to_numeric(df['sale_month'])
  df['sale_day'] = pd.to_numeric(df['sale_day'])
  # Don't need the garbage format date column, throw it away
  df.drop(columns=['date'], errors='ignore', inplace=True)

  # There are no rows with a sale time. Bruh.
  housing.drop(columns=['sale_time'], errors='ignore', inplace=True)

  return df

def add_sqft_columns(dataframe):
  df = dataframe.copy()
  # Get ratio of living space to lot space for individual house and neighboring houses and add to new columns
  df['sqft_living_vs_lot']           = df['sqft_living']   / df['sqft_lot']
  df['sqft_living_vs_lot_neighbors'] = df['sqft_living15'] / df['sqft_lot15']

  # Get ratio of living and lot space compared to neighboring houses and add to new columns
  df['sqft_living_vs_neighbors'] = df['sqft_living'] / df['sqft_living15']
  df['sqft_lot_vs_neighbors']    = df['sqft_lot']    / df['sqft_lot15']
  df['sqft_living_vs_lot_vs_neighbors'] = df['sqft_living_vs_lot'] / df['sqft_living_vs_lot_neighbors']

  return df

def add_recent_reno(dataframe):
  df = dataframe.copy()
  # Determine last renovation, or build date if not renovated
  df['last_update_year'] = np.where(df['yr_renovated'] > 0, df['yr_renovated'], df['yr_built'])
  # Calculate years since last renovation ("update" from above)
  df['years_since_renovation'] = df['sale_year'] - df['last_update_year']
  # Don't need the intermediate column, throw it away
  df.drop(columns=['last_update_year'], errors='ignore', inplace=True)

  return df

def add_neighboring_house_info(dataframe):
  df = dataframe.copy()
  # Extract lat & long to build the tree
  coords = df[['lat', 'long']].values

  # KDTree helps with efficient nearest-neighbor search
  tree = KDTree(coords)

  # Find the k nearest neighbors for each house
  def find_nearest_neighbors(index, k=3):
      # Grab the coordinates of the current house
      current_point = coords[index]

      # Query KDTree for k+1 nearest neighbors (k+1 to include itself)
      distances, indices = tree.query(current_point, k=k+1)

      # Exclude itself (first entry) from results
      return distances[1:], indices[1:]

  # How many neighbors to find?
  k_neighbors = 3

  neighbor_distances = []
  neighbor_prices = []

  for i in range(len(df)):
      distances, indices = find_nearest_neighbors(i, k=k_neighbors)

      # Store average distance to k neighbors
      neighbor_distances.append(distances.mean())

      # Store average price of k closest neighbors
      avg_price = df.iloc[indices]['price'].mean()
      neighbor_prices.append(avg_price)

  # These are in degrees, not km
  df['avg_distance_to_neighbors'] = neighbor_distances
  df['avg_price_of_neighbors']    = neighbor_prices



  # Count houses within a given radius (e.g., 1 km)
  def count_houses_within_radius(index, radius_km=1.0):
      current_point = coords[index]

      # Query KDTree for all neighbors within the specified radius
      indices = tree.query_ball_point(current_point, r=radius_km / 111)  # Convert km to degrees (~111 km per degree latitude)

      # Exclude itself (first entry) —— (*, 0) ensures non-negative count
      return max(len(indices) - 1, 0)

  # Compute the count for each house
  houses_within_1km = [count_houses_within_radius(i, radius_km=1.0) for i in range(len(df))]

  # Add the new column to the DataFrame
  df['houses_within_1km'] = houses_within_1km

  return df

In [6]:
len7_ids  = housing[housing['id'].astype(str).str.len() ==  7].copy()
len8_ids  = housing[housing['id'].astype(str).str.len() ==  8].copy()
len9_ids  = housing[housing['id'].astype(str).str.len() ==  9].copy()
len10_ids = housing[housing['id'].astype(str).str.len() == 10].copy()

# len7_ids.info()  #     17 rows
# len8_ids.info()  #    108 rows
# len9_ids.info()  #  1,787 rows
# len10_ids.info() # 18,088 rows (the rest of them)

# I have no idea what this means but it's interesting

In [7]:
ghost_floors = housing[housing['floors'] == 1][housing['sqft_basement'] > 0].copy()

# There aren't any floors with 0 sqft aboveground, so all "ghost floors" are definitely split.
'''
maybe = housing[housing['sqft_above'] == 0].copy()
maybe.info()
'''

ghost_floors.info() # 5,083 entries marked as one floor, but with sqft_living split into sqft_above and sqft_basement? Can't be an accident.

<class 'pandas.core.frame.DataFrame'>
Index: 5083 entries, 1 to 19983
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             5083 non-null   int64  
 1   date           5083 non-null   object 
 2   bedrooms       5083 non-null   int64  
 3   bathrooms      5083 non-null   float64
 4   sqft_living    5083 non-null   int64  
 5   sqft_lot       5083 non-null   int64  
 6   floors         5083 non-null   float64
 7   waterfront     5083 non-null   int64  
 8   view           5083 non-null   int64  
 9   condition      5083 non-null   int64  
 10  grade          5083 non-null   int64  
 11  sqft_above     5083 non-null   int64  
 12  sqft_basement  5083 non-null   int64  
 13  yr_built       5083 non-null   int64  
 14  yr_renovated   5083 non-null   int64  
 15  zipcode        5083 non-null   int64  
 16  lat            5083 non-null   float64
 17  long           5083 non-null   float64
 18  sqft_living1

<ipython-input-7-6187ee116660>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ghost_floors = housing[housing['floors'] == 1][housing['sqft_basement'] > 0].copy()


In [14]:
##### This cell is redundant because it's now implemented into split_sale_column().

sale_dates_with_times = housing[housing['sale_time'] != '000000'].copy()
# sale_dates_with_times.info()
# There are no rows with a sale time. Bruh.
housing.drop(columns=['sale_time'], errors='ignore', inplace=True)
housing.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sale_day,sqft_living_vs_lot,sqft_living_vs_lot_neighbors,sqft_living_vs_neighbors,sqft_lot_vs_neighbors,sqft_living_vs_lot_vs_neighbors,years_since_renovation,avg_distance_to_neighbors,avg_price_of_neighbors,houses_within_1km
0,1565930130,4,3.25,3760,4675,2.0,0,0,3,8,...,4,0.804278,0.813290,1.146341,1.159187,0.988919,7,0.002207,264983.333333,18
1,3279000420,3,1.75,1460,7800,1.0,0,0,2,7,...,15,0.187179,0.166561,1.114504,0.991736,1.123791,36,0.001008,234333.333333,63
2,194000575,4,1.00,1340,5800,1.5,0,2,3,7,...,14,0.231034,0.327586,0.705263,1.000000,0.705263,100,0.000807,482000.000000,108
3,2115510160,3,1.75,1440,8050,1.0,0,0,3,8,...,8,0.178882,0.239049,0.804469,1.075053,0.748306,29,0.001452,282500.000000,34
4,7522500005,2,1.50,1780,4750,1.0,0,0,4,7,...,15,0.374737,0.283462,1.053254,0.796713,1.322001,67,0.000400,853333.666667,114


In [9]:
'''
The goal of this box is to provide some housing density metrics, based on lat/long coordinates.
This does NOT give an accurate representation based on the total number of houses that exist neighboring a certain house,
    it only calculates based on those that were sold and tracked in this database.
The idea is that even though it doesn't represent reality, it might still give extra information
    based on the comparison to the same metric for other houses in the dataset.
(i.e. if 10 houses within 1km of house A were sold, compared to 100 within 1km of house B,
    then house B is probably in a more densely populated area... or so we hope.)
'''

# Implemented in functions in above cell
'''
# Extract lat & long to build the tree
coords = housing[['lat', 'long']].values

# KDTree helps with efficient nearest-neighbor search
tree = KDTree(coords)

# Find the k nearest neighbors for each house
def find_nearest_neighbors(index, k=3):
    # Grab the coordinates of the current house
    current_point = coords[index]

    # Query KDTree for k+1 nearest neighbors (k+1 to include itself)
    distances, indices = tree.query(current_point, k=k+1)

    # Exclude itself (first entry) from results
    return distances[1:], indices[1:]

# How many neighbors to find?
k_neighbors = 3

neighbor_distances = []
neighbor_prices = []

for i in range(len(housing)):
    distances, indices = find_nearest_neighbors(i, k=k_neighbors)

    # Store average distance to k neighbors
    neighbor_distances.append(distances.mean())

    # Store average price of k closest neighbors
    avg_price = housing.iloc[indices]['price'].mean()
    neighbor_prices.append(avg_price)

# These are in degrees, not km
housing['avg_distance_to_neighbors'] = neighbor_distances
housing['avg_price_of_neighbors']    = neighbor_prices



# Count houses within a given radius (e.g., 1 km)
def count_houses_within_radius(index, radius_km=1.0):
    current_point = coords[index]

    # Query KDTree for all neighbors within the specified radius
    indices = tree.query_ball_point(current_point, r=radius_km / 111)  # Convert km to degrees (~111 km per degree latitude)

    # Exclude itself (first entry) —— (*, 0) ensures non-negative count
    return max(len(indices) - 1, 0)

# Compute the count for each house
houses_within_1km = [count_houses_within_radius(i, radius_km=1.0) for i in range(len(housing))]

# Add the new column to the DataFrame
housing['houses_within_1km'] = houses_within_1km
'''

"\n# Extract lat & long to build the tree\ncoords = housing[['lat', 'long']].values\n\n# KDTree helps with efficient nearest-neighbor search\ntree = KDTree(coords)\n\n# Find the k nearest neighbors for each house\ndef find_nearest_neighbors(index, k=3):\n    # Grab the coordinates of the current house\n    current_point = coords[index]\n\n    # Query KDTree for k+1 nearest neighbors (k+1 to include itself)\n    distances, indices = tree.query(current_point, k=k+1)\n\n    # Exclude itself (first entry) from results\n    return distances[1:], indices[1:]\n\n# How many neighbors to find?\nk_neighbors = 3\n\nneighbor_distances = []\nneighbor_prices = []\n\nfor i in range(len(housing)):\n    distances, indices = find_nearest_neighbors(i, k=k_neighbors)\n\n    # Store average distance to k neighbors\n    neighbor_distances.append(distances.mean())\n\n    # Store average price of k closest neighbors\n    avg_price = housing.iloc[indices]['price'].mean()\n    neighbor_prices.append(avg_price)\

In [10]:
housing.head()
# with pd.option_context('display.float_format', '{:.5f}'.format):
#   print(housing.loc[housing['houses_within_1km'].idxmin()])

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,1565930130,20141104T000000,4,3.25,3760,4675,2.0,0,0,3,...,2740,1020,2007,0,98038,47.3862,-122.048,3280,4033,429900.0
1,3279000420,20150115T000000,3,1.75,1460,7800,1.0,0,0,2,...,1040,420,1979,0,98023,47.3035,-122.382,1310,7865,233000.0
2,194000575,20141014T000000,4,1.00,1340,5800,1.5,0,2,3,...,1340,0,1914,0,98116,47.5658,-122.389,1900,5800,455000.0
3,2115510160,20141208T000000,3,1.75,1440,8050,1.0,0,0,3,...,1440,0,1985,0,98023,47.3187,-122.390,1790,7488,258950.0
4,7522500005,20140815T000000,2,1.50,1780,4750,1.0,0,0,4,...,1080,700,1947,0,98117,47.6859,-122.395,1690,5962,555000.0


In [12]:
housing = split_sale_column(housing)
housing = add_sqft_columns(housing)
housing = add_recent_reno(housing)
housing = add_neighboring_house_info(housing)
housing.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sale_time,sqft_living_vs_lot,sqft_living_vs_lot_neighbors,sqft_living_vs_neighbors,sqft_lot_vs_neighbors,sqft_living_vs_lot_vs_neighbors,years_since_renovation,avg_distance_to_neighbors,avg_price_of_neighbors,houses_within_1km
0,1565930130,4,3.25,3760,4675,2.0,0,0,3,8,...,000000,0.804278,0.813290,1.146341,1.159187,0.988919,7,0.002207,264983.333333,18
1,3279000420,3,1.75,1460,7800,1.0,0,0,2,7,...,000000,0.187179,0.166561,1.114504,0.991736,1.123791,36,0.001008,234333.333333,63
2,194000575,4,1.00,1340,5800,1.5,0,2,3,7,...,000000,0.231034,0.327586,0.705263,1.000000,0.705263,100,0.000807,482000.000000,108
3,2115510160,3,1.75,1440,8050,1.0,0,0,3,8,...,000000,0.178882,0.239049,0.804469,1.075053,0.748306,29,0.001452,282500.000000,34
4,7522500005,2,1.50,1780,4750,1.0,0,0,4,7,...,000000,0.374737,0.283462,1.053254,0.796713,1.322001,67,0.000400,853333.666667,114


In [15]:
# First run to see what the above has done

unedited_X = housing_unedited.drop(columns=['price', 'date'])
unedited_y = housing_unedited['price']

edited_X = housing.drop(columns=['price'])
edited_y = housing['price']

unedited_reg = XGBRegressor()
edited_reg = XGBRegressor()

unedited_rf_reg = XGBRFRegressor()
edited_rf_reg = XGBRFRegressor()

unedited_X_train, unedited_X_test, unedited_y_train, unedited_y_test = train_test_split(unedited_X, unedited_y, test_size=0.2, random_state=42)
edited_X_train, edited_X_test, edited_y_train, edited_y_test = train_test_split(edited_X, edited_y, test_size=0.2, random_state=42)

unedited_reg.fit(unedited_X_train, unedited_y_train)
edited_reg.fit(edited_X_train, edited_y_train)

unedited_rf_reg.fit(unedited_X_train, unedited_y_train)
edited_rf_reg.fit(edited_X_train, edited_y_train)

unedited_tree_predictions = unedited_reg.predict(unedited_X_test)
edited_tree_predictions = edited_reg.predict(edited_X_test)

unedited_rf_predictions = unedited_rf_reg.predict(unedited_X_test)
edited_rf_predictions = edited_rf_reg.predict(edited_X_test)

unedited_tree_rmse = root_mean_squared_log_error(unedited_y_test, unedited_tree_predictions)
edited_tree_rmse = root_mean_squared_log_error(edited_y_test, edited_tree_predictions)

unedited_rf_rmse = root_mean_squared_log_error(unedited_y_test, unedited_rf_predictions)
edited_rf_rmse = root_mean_squared_log_error(edited_y_test, edited_rf_predictions)

print(f"Unedited Tree RMSLE: {unedited_tree_rmse}")
print(f"Edited Tree RMSLE: {edited_tree_rmse}")
print(f"Unedited RF RMSLE: {unedited_rf_rmse}")
print(f"Edited RF RMSLE: {edited_rf_rmse}")

# That's really awkward, this is actually terrible.

Unedited Tree RMSE: 0.17671965859001126
Edited Tree RMSE: 0.17171011607561842
Unedited RF RMSE: 0.23076756126010903
Edited RF RMSE: 0.21140538455186228


In [ ]:
mini_holdout = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test_mini.csv')
mini_holdout.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,930000305,20141110T000000,4,1.75,2120,7680,1.0,0,0,4,7,1060,1060,1950,0,98177,47.7172,-122.361,1530,7680
1,9541600015,20150211T000000,4,2.25,2010,15375,1.0,0,0,4,8,2010,0,1957,0,98005,47.5956,-122.174,1930,15375
2,7338000150,20150129T000000,2,1.00,1070,4200,1.0,0,0,4,6,1070,0,1983,0,98002,47.3336,-122.215,1150,4200
3,6113400046,20140723T000000,4,2.50,1890,15770,2.0,0,0,4,7,1890,0,1968,0,98166,47.4281,-122.343,2410,15256
4,291310170,20140804T000000,3,2.50,1600,2610,2.0,0,0,3,8,1600,0,2005,0,98027,47.5344,-122.068,1445,1288
